## Using rENA in python

The example below shows how to use the R library rENA with the python language.
All python libraries can be used to manipulate and arrange data for plotting.

All library functions can be called using the rENA interface. Another detail is that dots (.) should be replaced by underscores (_).

Example:
```
    The ena.plot() function can be called using rENA.ena_plot()
    The ena.plot.points() function can be called using rENA.ena_plot_points().
```
    
### Necessary libraries:
The R libraries rENA and data.table must be installed using the commands below:
```
install.packages("rENA")
install.packages("data.table")
```
    
In python we must install the rpy2 library using the following command:
```
pip install rpy2
``` 

In [1]:
# -*- coding: UTF-8 -*-

import pandas as pd
from rpy2.robjects import r, pandas2ri, Formula
from rpy2.robjects.packages import importr
import rpy2.robjects as ro
from rpy2.robjects.conversion import localconverter
pandas2ri.activate()  # makes some conversions automatic

ModuleNotFoundError: No module named 'rpy2'

## Importing the rENA library from R
Its functions will be available in the "rENA" variable

In [2]:
rENA = importr('rENA')

## The robjects function makes it possible to use R functions inside python

In [3]:
# R functions ready to be called in python
pr = ro.r("print")
colMeans = ro.r("colMeans")
rmax = ro.r("max")

In [4]:
# Reading the data to be plotted
#rsdata = pd.read_csv("rsdata.csv")
# load your file
object = ro.r['load']('RS.data.rda')
r.data('RS.data')
rsdata = r['RS.data']

## Identify columns to use in the accumulator
Before running the ena_accumulate_data function, we first need to identify which data columns to use in our units, conversations and codes. There is also an optional metadata parameter (meta), which is for unit-specific data that we want to carry through the accumulation process and keep associated with the identified units.

In [5]:
units = rsdata[['Condition','UserName']]
conversation = rsdata[['Condition','GroupName']]
codes = rsdata[['Data','Technical.Constraints','Performance.Parameters','Client.and.Consultant.Requests','Design.Reasoning','Collaboration']]
meta = rsdata[["CONFIDENCE.Change","CONFIDENCE.Pre","CONFIDENCE.Post","C.Change"]]

## Performing data accumulation
With the data identified and subsets, the accumulation and generation of sets is quite simple.

In [6]:
accum = rENA.ena_accumulate_data(units, conversation, codes, meta)

## Generating the ENA set
The most basic form of an ENAset can be generated by passing the result of calling the ena_accumulate_data function to ena_make_set.

In [7]:
set = rENA.ena_make_set(
  enadata=accum
)

## Plot Units In Each Group

In [8]:
points = (set.rx2('points'))

df_points1 = points.loc[points['Condition'] == "FirstGame"]
df_points2 = points.loc[points['Condition'] == "SecondGame"]
### Subset rotated points for the first condition
first_game_points = df_points1.drop(columns=['ENA_UNIT','Condition','UserName', 'CONFIDENCE.Change', 'CONFIDENCE.Pre', 'CONFIDENCE.Post', "C.Change"])
### Subset rotated points for the second condition
second_game_points = df_points2.drop(columns=['ENA_UNIT','Condition','UserName',  'CONFIDENCE.Change', 'CONFIDENCE.Pre', 'CONFIDENCE.Post', "C.Change"])



plot = rENA.ena_plot(set, scale_to ="network", title ="Groups of Units")
plot = rENA.ena_plot_points(plot, points = first_game_points, confidence_interval ="box", colors = ("blue"))
plot = rENA.ena_plot_points(plot, points = second_game_points, confidence_interval ="box", colors = ("red"))
pr(plot)

R object with classes: ('ENAplot', 'R6') mapped to:

## Plotting Means

In [9]:
plot = rENA.ena_plot(set, scale_to = [-1, 0, 1], title ="Groups and Means")
plot = rENA.ena_plot_points(plot, points = first_game_points, confidence_interval ="box", colors = ("blue"))
plot = rENA.ena_plot_points(plot, points = second_game_points, confidence_interval ="box", colors = ("red"))
plot = rENA.ena_plot_group(plot, first_game_points, colors = ("red"), confidence_interval ="box")
plot = rENA.ena_plot_group(plot, second_game_points, colors =("blue"), confidence_interval ="box")
pr(plot)

R object with classes: ('ENAplot', 'R6') mapped to:

## Plotting a Network

In [10]:
line_weights = (set.rx2('line.weights'))

### Subset lineweights for SecondGame and Calculate the colMeans
first_game_lineweights = line_weights.loc[points['Condition'] == "FirstGame"]
second_game_lineweights = line_weights.loc[points['Condition'] == "SecondGame"]

first_game_lineweights = first_game_lineweights.drop(columns=['ENA_UNIT','Condition','UserName', 'CONFIDENCE.Change', 'CONFIDENCE.Pre', 'CONFIDENCE.Post', "C.Change"])
### Subset rotated points for the second condition
second_game_lineweights = second_game_lineweights.drop(columns=['ENA_UNIT','Condition','UserName',  'CONFIDENCE.Change', 'CONFIDENCE.Pre', 'CONFIDENCE.Post', "C.Change"])

first_game_mean = colMeans(first_game_lineweights)
second_game_mean = colMeans(second_game_lineweights)

subtracted_mean = first_game_mean - second_game_mean

In [11]:
plot_first = rENA.ena_plot(set, title ="FirstGame")
plot_first = rENA.ena_plot_network(plot_first, network = first_game_mean)
pr(plot_first)

R object with classes: ('ENAplot', 'R6') mapped to:

In [12]:
plot_second = rENA.ena_plot(set, title ="SecondGame")
plot_second = rENA.ena_plot_network(plot_second, network = second_game_mean, colors = ("blue"))
pr(plot_second)

R object with classes: ('ENAplot', 'R6') mapped to:

## Plot subtracted network only

In [13]:
plot_sub = rENA.ena_plot(set, title ="Subtracted")
plot_sub = rENA.ena_plot_network(plot_sub, network = subtracted_mean)
pr(plot_sub)

R object with classes: ('ENAplot', 'R6') mapped to:

## Plot Everything Together

In [14]:
rotation = (set.rx2('rotation'))
nodes = rotation.rx2('nodes')

# Scale the nodes to match that of the network, for better viewing
first_game_points_max = rmax(first_game_points)
second_game_points_max = rmax(second_game_points)
if(first_game_points_max> second_game_points_max):
  point_max = first_game_points_max
else:
  point_max = second_game_points_max

nodes = nodes.drop(columns=['code'])
max_nodes = rmax(nodes)
with localconverter(ro.default_converter + pandas2ri.converter):
  first_game_points = ro.conversion.py2rpy(first_game_points)

first_game_scaled = first_game_points
second_game_scaled = second_game_points

plot = rENA.ena_plot(set, title ="Plot with Units and Network", font_family ="Times")
plot = rENA.ena_plot_points(plot, points = first_game_scaled, colors = ("red"))
plot = rENA.ena_plot_points(plot, points = second_game_scaled, colors = ("blue"))
plot = rENA.ena_plot_group(plot, point = first_game_scaled, colors =("red"), confidence_interval ="box")
plot = rENA.ena_plot_group(plot, point = second_game_scaled, colors =("blue"), confidence_interval ="box")
plot = rENA.ena_plot_network(plot, network = subtracted_mean)
pr(plot)


R object with classes: ('ENAplot', 'R6') mapped to: